In [1]:
from stockPriceForecast_7Days import SPF_7Days
from prepareDataset_7Days import PD_7Days
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
emiten_data = pd.read_csv('BBCA.csv')
emiten_data.set_index('Date', inplace=True)
pd_7days = PD_7Days(emiten_data)

In [3]:
feature_data = pd_7days.create_feature_data()
target_data = pd_7days.create_target_data()
spf_7days = SPF_7Days(feature_data, target_data)

In [4]:
training_perc = 0.95
spf_7days.slice_training_testing_dataset(training_perc)

In [5]:
model = spf_7days.create_model()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 6, 64)          │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 6, 40)          │         4,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 24)             │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,193 (39.82 KB)

 Trainable params: 10,193 (39.82 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
learning_rate = 0.05
epochs = 250

model = spf_7days.model_fitting(model, learning_rate, epochs)

Epoch 1/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6912 - mae: 1.1188
Epoch 2/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1858 - mae: 0.5337 
Epoch 3/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0831 - mae: 0.3375 
Epoch 4/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0392 - mae: 0.2244 
Epoch 5/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0292 - mae: 0.1951 
Epoch 6/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0254 - mae: 0.1762 
Epoch 7/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0224 - mae: 0.1657 
Epoch 8/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0220 - mae: 0.1638 
Epoch 9/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0251 - mae: 0.1822 
Epoch 10/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0235 - mae: 0.1670 
Epoch 11/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0226 - mae: 0.1709 
Epoch 12/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0196 - mae: 0.1609 
Epoch 13/250
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 

In [7]:
spf_7days.save_model_in_keras(model, 'BBCA')

In [8]:
forecast_feature = spf_7days.slice_forecast_dataset()

In [9]:
test_forecast = spf_7days.model_forecast(spf_7days.test_feature, 'BBCA')
train_forecast = spf_7days.model_forecast(spf_7days.train_feature, 'BBCA')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


In [10]:
def measure_accuracy(actual, forecast):
    incrementals_forecast = (100 * (forecast[1:] / forecast[:-1])) - 100
    incrementals_actual = (100 * (actual[1:] / actual[:-1])) - 100
    if len(incrementals_actual) != len(incrementals_forecast):
        raise Exception("There is something wrong with your data")
    
    return np.sum(incrementals_forecast * incrementals_actual > 0) / len(incrementals_actual)

In [12]:
measure_accuracy(spf_7days.test_target, test_forecast)

0.5714285714285714

In [13]:
measure_accuracy(spf_7days.train_target, train_forecast)

/var/folders/5v/1b7c89d50b3f69z5x1qgtr180000gn/T/ipykernel_30960/415465749.py:3: RuntimeWarning: divide by zero encountered in divide
  incrementals_actual = (100 * (actual[1:] / actual[:-1])) - 100


0.39849624060150374